In [1]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
import meshio
import numpy as np
gmsh.initialize()

In [2]:
def read_spheres_position_file(spheres_position_path):
    """
    Reads file input that contains the centers of spheres. The path is assumed to have
    been generated using code from Skoge et al.
    """
    centers = []
    radius = 0
    n = 0
    with open(spheres_position_path) as fp:
        for i, row in enumerate(fp.readlines()):
            if i < 2:
                continue
            if i == 2:
                n = int(row)
            if i == 3:
                radius = float(row)
            if i < 6:
                continue
            x, y, z, _ = row.split(' ')
            centers.append((float(x), float(y), float(z)))
    return centers, float(radius)/2, n


In [3]:
gmsh.model.add("3D")
Lx, Ly, Lz = 1, 1, 1
resolution = 0.001
channel = gmsh.model.occ.addBox(0, 0, 0, Lx, Ly, Lz)

In [4]:
spheres_ = []
centers, r, n_spheres = read_spheres_position_file("/home/leshinka/spheres/write.dat")
for center in centers:
    x, y, z = center
    sphere = gmsh.model.occ.addSphere(*center, r)
    spheres_.append(sphere)
channel = gmsh.model.occ.cut([(3, channel)], [(3, sphere) for sphere in spheres_])

In [5]:
gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)

In [6]:
marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], marker)
gmsh.model.setPhysicalName(volumes[0][0], marker, "conductor")

In [7]:
surfaces = gmsh.model.occ.getEntities(dim=2)
left_marker = 1
right_marker = 3
sphere_marker = 5
spheres = []
walls = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [0, Ly/2, Lz/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], left_marker)
        left = surface[1]
        gmsh.model.setPhysicalName(surface[0], left_marker, "left")
    elif np.allclose(com, [Lx, Ly/2, Lz/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], right_marker)
        gmsh.model.setPhysicalName(surface[0], right_marker, "right")
        right = surface[1]
    elif np.isclose(com[2], 0) or np.isclose(com[1], Ly) or np.isclose(com[2], Lz) or np.isclose(com[1], 0):
        walls.append(surface[1])
    else:
        spheres.append(surface[1])
gmsh.model.addPhysicalGroup(2, spheres, sphere_marker)
gmsh.model.setPhysicalName(2, sphere_marker, "sphere")

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "FacesList", spheres)

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", resolution)
gmsh.model.mesh.field.setNumber(2, "LcMax", 20*resolution)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.5*r)
gmsh.model.mesh.field.setNumber(2, "DistMax", r)

# gmsh.model.mesh.field.add("Distance", 3)
# gmsh.model.mesh.field.setNumbers(3, "FacesList", [left])
# gmsh.model.mesh.field.add("Threshold", 4)
# gmsh.model.mesh.field.setNumber(4, "IField", 3)
# gmsh.model.mesh.field.setNumber(4, "LcMin", 5*resolution)
# gmsh.model.mesh.field.setNumber(4, "LcMax", 10*resolution)
# gmsh.model.mesh.field.setNumber(4, "DistMin", 0.1)
# gmsh.model.mesh.field.setNumber(4, "DistMax", 0.5)

gmsh.model.mesh.field.add("Min", 5)
gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2])
gmsh.model.mesh.field.setAsBackgroundMesh(5)

In [8]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 20 (Circle)
Info    : [ 10%] Meshing curve 23 (Circle)
Info    : [ 10%] Meshing curve 32 (Circle)
Info    : [ 10%] Meshing curve 38 (Circle)
Info    : [ 10%] Meshing curve 44 (Circle)
Info    : [ 10%] Meshing curve 47 (Circle)
Info    : [ 10%] Meshing curve 59 (Circle)
Info    : [ 20%] Meshing curve 62 (Circle)
Info    : [ 20%] Meshing curve 74 (Circle)
Info    : [ 20%] Meshing curve 77 (Circle)
Info    : [ 20%] Meshing curve 86 (Circle)
Info    : [ 20%] Meshing curve 95 (Circle)
Info    : [ 20%] Meshing curve 101 (Circle)
Info    : [ 20%] Meshing curve 128 (Circle)
Info    : [ 30%] Meshing curve 131 (Circle)
Info    : [ 30%] Meshing curve 140 (Circle)
Info    : [ 30%] Meshing curve 146 (Circle)
Info    : [ 30%] Meshing curve 149 (Circle)
Info    : [ 30%] Meshing curve 151 (Line)
Info    : [ 30%] Meshing curve 152 (Line)
Info    : [ 30%] Meshing curve 153 (Line)
Info    : [ 30%] Meshing curve 154 (Circle)
Info    : [ 30%] Meshing c

In [ ]:
gmsh.write("mesh3D.msh")

Info    : Writing 'mesh3D.msh'...
Info    : Done writing 'mesh3D.msh'


In [ ]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [ ]:
mesh_3d = meshio.read("mesh3D.msh")
tetrahedral_mesh = create_mesh(mesh_3d, "tetra")
meshio.write("mesh/1_0_1/mesh_tetr.xdmf", tetrahedral_mesh)

In [ ]:
cmd_3d = 'mpirun -n 2 python3 ion_transport.py --working_dir=/home/leshinka/dev/ssb/ --grid_info=1_0_1 --file_shape=1_1'
!{cmd_3d}

done loading tetrahedral mesh
done loading tetrahedral mesh
setting problem..
setting problem..
solving problem..
solving problem..
--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpirun noticed that process rank 0 with PID 0 on node leshinka-ThinkPad-T15-Gen-2i exited on signal 9 (Killed).
--------------------------------------------------------------------------
